In [1]:
from utils import *
from CNN import *
from random import *
import copy
import numpy as np
import cv2
import os

# Load data

In [2]:
# load image and label from folder
def load_images_from_folder(folder):
    images = []
    labels = []
    for label in os.listdir(folder):
        label_path = os.path.join(folder, label)
        if os.path.isdir(label_path):
            for filename in os.listdir(label_path):
                img_path = os.path.join(label_path, filename)
                if os.path.isfile(img_path):
                    image = cv2.imread(img_path)
                    if image is not None:
                        images.append(image)
                        labels.append(label)  
    return images, labels

In [3]:
trainDir = '../data/archive/PandasBears/Train/'
testDir = '../data/archive/PandasBears/Test/'

train_x, train_y = load_images_from_folder(trainDir)
test_x, test_y = load_images_from_folder(trainDir)

In [25]:
train_y_encoded = [1 if item == 'Bears' else 0 for item in train_y]

# Ncross Val

In [4]:
def n_fold_cross_validation(data, labels, n_folds):
    if len(data) != len(labels):
        print("data and label is not the same length")
        return 
    fold_data = []
    data_labels = list(zip(data, labels))
    np.random.shuffle(data_labels)

    fold_size = len(data) // n_folds

    for i in range(n_folds):
        start = i * fold_size
        end = (i + 1) * fold_size if i < n_folds - 1 else len(data)
        
        test_set = data_labels[start:end]
        train_set = data_labels[:start] + data_labels[end:]
        
        fold_data.append((train_set, test_set))

    return fold_data

In [26]:
n_folds = 10

folds = n_fold_cross_validation(train_x, train_y_encoded, n_folds)

In [27]:
len(folds)

10

In [28]:
len(folds[0])

2

In [29]:
len(folds[0][0])

450

In [30]:
folds[0][0][0][1]

0

In [31]:

folds[0][0][0][0].shape

(256, 256, 3)

In [32]:
# train_set array of 

In [33]:
for i, (train_set, test_set) in enumerate(folds):
    print(train_set[0][0])
    print(f"Fold {i+1} - Train Set Length: {len(train_set)}, Test Set Length: {len(test_set)}")

[[[ 40 129  86]
  [ 38 127  84]
  [ 35 124  80]
  ...
  [ 18 108  65]
  [ 19 108  65]
  [ 20 109  66]]

 [[ 39 128  85]
  [ 38 127  84]
  [ 35 124  80]
  ...
  [ 22 112  69]
  [ 24 113  70]
  [ 25 114  71]]

 [[ 37 126  83]
  [ 36 125  82]
  [ 34 123  80]
  ...
  [ 23 115  72]
  [ 27 117  74]
  [ 29 119  76]]

 ...

 [[125 147 175]
  [122 144 172]
  [112 136 164]
  ...
  [ 10  16  21]
  [  4   8  13]
  [  0   1   6]]

 [[135 157 185]
  [131 153 181]
  [142 166 196]
  ...
  [ 14  20  25]
  [ 10  14  19]
  [  8  12  17]]

 [[129 151 179]
  [107 129 157]
  [110 134 164]
  ...
  [ 16  25  29]
  [ 22  26  31]
  [ 31  35  40]]]
Fold 1 - Train Set Length: 450, Test Set Length: 50
[[[ 37 133  92]
  [ 35 131  90]
  [ 32 128  87]
  ...
  [ 76 127 100]
  [ 80 131 104]
  [ 85 136 109]]

 [[ 35 131  90]
  [ 33 129  88]
  [ 31 127  86]
  ...
  [ 79 130 103]
  [ 83 134 107]
  [ 87 138 111]]

 [[ 35 129  88]
  [ 34 128  87]
  [ 33 127  86]
  ...
  [ 81 132 105]
  [ 85 136 109]
  [ 89 140 113]]

 ...



In [34]:
def evaluate(label, prediction):
    if len(label) != len(prediction):
        print("Label and prediction have differenct length")
        return

    correct = 0
    true_positives = 0
    false_positives = 0
    false_negatives = 0
    true_negative = 0

    for i in range(len(label)):
        if label[i] == prediction[i]:
            correct += 1
            if label[i] == 1:
                true_positives += 1
            else:
                true_negative +=0
        else:
            if label[i] == 1:
                false_negatives += 1
            else:
                false_positives += 1

    accuracy = correct / len(label)

    precision = true_positives / (true_positives + false_positives) if (true_positives + false_positives) > 0 else 0
    recall = true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) > 0 else 0
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

    metrics = {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1_score": f1_score,
        "confusion_matrix" : [true_positives, false_positives, false_negatives, true_negative]
    }

    return metrics


In [51]:
model_list = []
metric_list = []
for i, (train_set, test_set) in enumerate(folds):
    model = CNN(input_shape_=folds[0][0][0][0].shape)
    model.add(ConvolutionLayer(8, [3,3], 0, (1,1)))
    model.add(DetectorLayer())
    model.add(PoolingLayer((2,2), "max"))
    model.add(FlattenLayer())
    model.add(DenseLayer(8, "relu"))

    model.add(DenseLayer(1, "sigmoid"))
    model.compile()
    #   
    #  train_label= [(image,label), .....]
    data = []
    label = []
    for j in train_set:
        data.append(j[0])
        label.append(j[1])

    model.fit(np.array(data), label, 5)

    model_list.append(model)

    # output = model.predict(standard6x6x1)



FILTER
[[[1. 1. 1.]
  [1. 1. 1.]
  [1. 1. 1.]]

 [[1. 1. 1.]
  [1. 1. 1.]
  [1. 1. 1.]]

 [[1. 1. 1.]
  [1. 1. 1.]
  [1. 1. 1.]]

 [[1. 1. 1.]
  [1. 1. 1.]
  [1. 1. 1.]]

 [[1. 1. 1.]
  [1. 1. 1.]
  [1. 1. 1.]]

 [[1. 1. 1.]
  [1. 1. 1.]
  [1. 1. 1.]]

 [[1. 1. 1.]
  [1. 1. 1.]
  [1. 1. 1.]]

 [[1. 1. 1.]
  [1. 1. 1.]
  [1. 1. 1.]]]
[array([[[ 40, 129,  86],
        [ 38, 127,  84],
        [ 35, 124,  80],
        ...,
        [ 18, 108,  65],
        [ 19, 108,  65],
        [ 20, 109,  66]],

       [[ 39, 128,  85],
        [ 38, 127,  84],
        [ 35, 124,  80],
        ...,
        [ 22, 112,  69],
        [ 24, 113,  70],
        [ 25, 114,  71]],

       [[ 37, 126,  83],
        [ 36, 125,  82],
        [ 34, 123,  80],
        ...,
        [ 23, 115,  72],
        [ 27, 117,  74],
        [ 29, 119,  76]],

       ...,

       [[125, 147, 175],
        [122, 144, 172],
        [112, 136, 164],
        ...,
        [ 10,  16,  21],
        [  4,   8,  13],
        [  0,   1,

ValueError: Invalid input or filter size